### Goal

- For each gene in the genome, count the number of nearby CNEs within distance window (adjustable distance). 

### Input

- CNE coordinates
- GFF genome annotations converted to gffutils database

### Output

- all_sp_nearby_cnes.tsv: Data frame with basic stats for each species and distance threshold.  
- all_sp_nearby_cnes.pickle Dictionary of nearby CNEs to be processed with overrep_domains_nearby_cnes.ipynb

In [8]:
import pandas as pd
import glob
import json
import sys
import gffutils
from statistics import mean 
from os import listdir
from os.path import isfile, join
from collections import namedtuple
import pickle

#### Input data

In [15]:
coord_dir = "../../retrieve_original_coords/filtered_coords/"
gff_dir = "../../gff_dbs/"

#### Species list

In [10]:
species_list = ['apis',
                'aaeg',
                'aamp',
                'amel',
                'alic',
                'anas',
                'cscu',
                'cdip',
                'csec',
                'cfel',
                'dmel',
                'eaff',
                'gocc',
                'hazt',
                'lpol',
                'ahyp',
                'obir',
                'phum',
                'pvan',
                'ptri',
                'smim',
                'smar',
                'tpal',
                'tcas']

#### Distance threshold list

In [11]:
dist_threshold_list = [10000, 25000, 50000, 75000, 100000, 200000, 300000, 400000, 500000]

#### Function that calculates distance between two sequences

In [12]:
def seq_dist(start_1,end_1,start_2,end_2):
    return(abs(min(end_1,end_2) - max(start_1,start_2)))

###  Function to retrieve coordinates of each gene

In [19]:
def parse_cds(species):
    gff_file = gff_dir + species + "_gff.db"
    gene_db = gffutils.FeatureDB(gff_file,keep_order=True)
    cds_coords = {}
    gene_scaff_dict = {}
    for gene in gene_db.features_of_type(featuretype="gene"):
        if 'gene_biotype' not in gene.attributes or 'gene_biotype' in gene.attributes and \
            gene.attributes['gene_biotype'][0] == 'protein_coding' :
            scaff = gene.chrom
            gene_id = str(gene.id)
            gene_scaff_dict[gene_id] = scaff
            for CDS in gene_db.children(gene.id,featuretype='CDS'): 
                if species in ['spha', 'echl']:
                    CDS_id = CDS.id.split("-")[-1].split("_")[0]
                elif species in ['hruf']:
                    CDS_id = CDS.id.split(":")[0]
                elif species in ['agra']:
                    CDS_id = CDS.id.split("model.")[-1].split("_")[0]
                else:
                    CDS_id = "_".join(CDS.id.split("-")[-1].split("_")[:2])
                CDS_start = CDS.start
                CDS_end = CDS.end
                if gene_id not in cds_coords:
                    cds_coords[gene_id] = [CDS_start, CDS_end]
                else:
                    cds_coords[gene_id] = [min(CDS_start,cds_coords[gene_id][0]) , max(CDS_end,cds_coords[gene_id][1]) ]
    return(cds_coords, gene_scaff_dict)

In [20]:
cds_coords, gene_scaff_dict = parse_cds('lpol')

In [22]:
cds_coords

{'gene-LOC106456832': [2798, 21366],
 'gene-LOC106466742': [147538, 148950],
 'gene-LOC106477029': [351899, 360775],
 'gene-LOC106456833': [437378, 608322],
 'gene-LOC111085464': [608733, 619667],
 'gene-LOC106457832': [792808, 925197],
 'gene-LOC106457563': [956304, 998504],
 'gene-LOC106457663': [1046831, 1132952],
 'gene-LOC106457765': [1151353, 1177206],
 'gene-LOC106458816': [1195078, 1197294],
 'gene-LOC106459800': [1303279, 1306712],
 'gene-LOC106460795': [1486322, 1493673],
 'gene-LOC106478102': [1501969, 1604399],
 'gene-LOC111087465': [1623300, 1705067],
 'gene-LOC106461769': [1842501, 1896076],
 'gene-LOC106462746': [1985589, 2003262],
 'gene-LOC106478203': [2163205, 2201845],
 'gene-LOC111089463': [2219189, 2246770],
 'gene-LOC106458041': [2296038, 2359342],
 'gene-LOC106463729': [2428699, 2461553],
 'gene-LOC106458387': [2472424, 2602122],
 'gene-LOC106464713': [2613381, 2667802],
 'gene-LOC106478309': [2721156, 2740348],
 'gene-LOC106478541': [2751655, 2790344],
 'gene-LO

In [21]:
gene_scaff_dict

{'gene-LOC106456832': 'NW_013665610.1',
 'gene-LOC106466742': 'NW_013665610.1',
 'gene-LOC106477029': 'NW_013665610.1',
 'gene-LOC106456833': 'NW_013665610.1',
 'gene-LOC111085464': 'NW_013665610.1',
 'gene-LOC106457832': 'NW_013665610.1',
 'gene-LOC106457563': 'NW_013665610.1',
 'gene-LOC106457663': 'NW_013665610.1',
 'gene-LOC106457765': 'NW_013665610.1',
 'gene-LOC106458816': 'NW_013665610.1',
 'gene-LOC106459800': 'NW_013665610.1',
 'gene-LOC106460795': 'NW_013665610.1',
 'gene-LOC106478102': 'NW_013665610.1',
 'gene-LOC111087465': 'NW_013665610.1',
 'gene-LOC106461769': 'NW_013665610.1',
 'gene-LOC106462746': 'NW_013665610.1',
 'gene-LOC106478203': 'NW_013665610.1',
 'gene-LOC111089463': 'NW_013665610.1',
 'gene-LOC106458041': 'NW_013665610.1',
 'gene-LOC106463729': 'NW_013665610.1',
 'gene-LOC106458387': 'NW_013665610.1',
 'gene-LOC106464713': 'NW_013665610.1',
 'gene-LOC106478309': 'NW_013665610.1',
 'gene-LOC106478541': 'NW_013665610.1',
 'gene-LOC106465725': 'NW_013665610.1',


### Function to retrieve CNEs within distance windows

In [23]:
def find_nearby_cnes(cds_coords_dict, species_id, dist_threshold_list, gene_scaff_dict):
    # Create dict to hold results
    distances_to_cnes = {}
    for threshold in dist_threshold_list:
        distances_to_cnes[threshold] = {}
    # Read CNE coordinates
    cne_coords = pd.read_csv(coord_dir + species_id + '_orig_coords.tsv', sep="\t")
    print("Find nearby CNEs")
    # Loop through each gene
    for cds_name, cds_coord in cds_coords_dict.items():
        # Retrieve gene scaffold, ID, and coordinates
        for threshold in dist_threshold_list:
            distances_to_cnes [threshold][cds_name] = []
        scaffold = gene_scaff_dict[cds_name]
        cds_start = cds_coord[0]
        cds_end = cds_coord[1]
        # Set dist_to_cne variable to check for closer CNEs. Start with absurdly high value
        # Check if scaffold contains CNEs and if yes retrieve them
        if scaffold in list(cne_coords['scaffold']):
            cnes_on_scaff_df = cne_coords[cne_coords['scaffold'] == scaffold]
            for idx, cne in cnes_on_scaff_df.iterrows():
                # Calculate distance between each CNE on scaffold and current gene
                cne_start = cne['orig_start']
                cne_end = cne['orig_end']
                distance = seq_dist(cne_start,cne_end, cds_start, cds_end)
                # Retrieve CNE if within distance threshold
                for threshold in dist_threshold_list:
                    if distance < threshold:
                        distances_to_cnes[threshold][cds_name].append(cne['cne_id'])
    return(distances_to_cnes)

### Retrieve nearby CNEs for each gene and distance threshold + calculate stats

In [24]:
df = pd.DataFrame(columns=['species', 'distance_threshold', 'mean_nearby_cnes', 'min_nearby_cnes', 'max_nearby_cnes'])
all_species_nearby_cnes = {}
for species in species_list:
    print("species:" , species)    
    cds_coords, gene_scaff_dict = parse_cds(species)
    nearby_cnes = find_nearby_cnes(cds_coords, species, dist_threshold_list, gene_scaff_dict)
    all_species_nearby_cnes[species] = nearby_cnes
    for threshold, dist_dict in nearby_cnes.items():
        cne_counts = []
        mean_nearby_cnes = 0
        min_nearby_cnes = 0
        max_cne_counts = 0
        print("distance threshold: ", threshold)
        for gene, cne_list in dist_dict.items():
            cne_count = len(cne_list)
            cne_counts.append(cne_count)
        if len(cne_counts)>0:
            mean_nearby_cnes = round(mean(cne_counts), 2)
            min_nearby_cnes = min(cne_counts)
            max_cne_counts = max(cne_counts)
        row = [species, threshold, mean_nearby_cnes, min_nearby_cnes, max_cne_counts]
        df.loc[len(df), :] = row
        print(row)
# Write stats to file
df.to_csv("all_species_nearby_cnes.tsv", sep="\t", index=False)

species: apis
Find nearby CNEs
distance threshold:  10000
['apis', 10000, 0.04, 0, 4]
distance threshold:  25000
['apis', 25000, 0.08, 0, 4]
distance threshold:  50000
['apis', 50000, 0.14, 0, 4]
distance threshold:  75000
['apis', 75000, 0.2, 0, 4]
distance threshold:  100000
['apis', 100000, 0.26, 0, 5]
distance threshold:  200000
['apis', 200000, 0.5, 0, 7]
distance threshold:  300000
['apis', 300000, 0.74, 0, 8]
distance threshold:  400000
['apis', 400000, 0.96, 0, 11]
distance threshold:  500000
['apis', 500000, 1.18, 0, 14]
species: aaeg
Find nearby CNEs
distance threshold:  10000
['aaeg', 10000, 0.05, 0, 14]
distance threshold:  25000
['aaeg', 25000, 0.07, 0, 14]
distance threshold:  50000
['aaeg', 50000, 0.11, 0, 14]
distance threshold:  75000
['aaeg', 75000, 0.16, 0, 14]
distance threshold:  100000
['aaeg', 100000, 0.2, 0, 14]
distance threshold:  200000
['aaeg', 200000, 0.37, 0, 16]
distance threshold:  300000
['aaeg', 300000, 0.54, 0, 18]
distance threshold:  400000
['aaeg',

Find nearby CNEs
distance threshold:  10000
['obir', 10000, 1.27, 0, 251]
distance threshold:  25000
['obir', 25000, 2.58, 0, 260]
distance threshold:  50000
['obir', 50000, 5.23, 0, 280]
distance threshold:  75000
['obir', 75000, 8.28, 0, 304]
distance threshold:  100000
['obir', 100000, 11.6, 0, 313]
distance threshold:  200000
['obir', 200000, 26.58, 0, 353]
distance threshold:  300000
['obir', 300000, 42.28, 0, 408]
distance threshold:  400000
['obir', 400000, 58.7, 0, 439]
distance threshold:  500000
['obir', 500000, 75.44, 0, 515]
species: phum
Find nearby CNEs
distance threshold:  10000
['phum', 10000, 0.03, 0, 6]
distance threshold:  25000
['phum', 25000, 0.08, 0, 6]
distance threshold:  50000
['phum', 50000, 0.14, 0, 6]
distance threshold:  75000
['phum', 75000, 0.19, 0, 6]
distance threshold:  100000
['phum', 100000, 0.24, 0, 6]
distance threshold:  200000
['phum', 200000, 0.4, 0, 6]
distance threshold:  300000
['phum', 300000, 0.52, 0, 7]
distance threshold:  400000
['phum',

In [ ]:
### Write dictionary of gene-nearby CNEs to file

In [25]:
with open('all_species_nearby_cnes.pickle', 'wb') as handle:
    pickle.dump(all_species_nearby_cnes, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
#with open('all_species_nearby_cnes.pickle', 'rb') as handle:
#    all_species_nearby_cnes = pickle.load(handle)